In [1]:
import os
import numpy as np
from skimage import io, transform
from PIL import Image
import tensorflow as tf
import keras
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, LeakyReLU, Activation, Reshape, Input
from keras.layers import Conv2D, Conv2DTranspose, Lambda, Concatenate
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras import backend as K
from keras.backend.tensorflow_backend import set_session

K.set_learning_phase(False) 
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
def dick(x):
    x = K.expand_dims(x, axis=1)
    x = K.expand_dims(x, axis=2)
    x = K.tile(x, [1, 4, 4, 1])
    return x


def wasserstein(y_true, y_pred):

    # return K.mean(y_true * y_pred) / K.mean(y_true)
    return K.mean(y_true * y_pred)

class dcgan():
    def __init__(self):
        self.dis_depth = 64
        self.gen_depth = 512
        self.alpha = 0.2
        self.dim = 4
        self.batch = 64
        self.epochs = 300
        
    def getmodelweight(self):
    
        #generator weights
        
        truncate_normal = keras.initializers.TruncatedNormal(mean=0.0, stddev=0.02, seed=None)
        
        self.s_dense = Dense(256, kernel_initializer=truncate_normal, name='sd0')
        self.s_leak = LeakyReLU(self.alpha, name='slk')
        
        #4 4 512
        self.g_dense_1 = Dense(self.dim*self.dim*self.gen_depth, kernel_initializer=truncate_normal, name='gd1')
        self.g_conv_1 = Conv2DTranspose(int(self.gen_depth/2), 5, kernel_initializer=truncate_normal, strides=2, padding='same', name='gc1')
        self.g_conv_2 = Conv2DTranspose(int(self.gen_depth/4), 5, kernel_initializer=truncate_normal, strides=2, padding='same', name='gc2')
        self.g_conv_3 = Conv2DTranspose(int(self.gen_depth/8), 5, kernel_initializer=truncate_normal, strides=2, padding='same', name='gc3')
        self.g_conv_4 = Conv2DTranspose(3, 5, kernel_initializer=truncate_normal, strides=2, padding='same', name='gc4')
        
        self.g_batch_1 = BatchNormalization(momentum=0.9, epsilon=1e-5, name='gb1')
        self.g_batch_2 = BatchNormalization(momentum=0.9, epsilon=1e-5, name='gb2')
        self.g_batch_3 = BatchNormalization(momentum=0.9, epsilon=1e-5, name='gb3')
        self.g_batch_4 = BatchNormalization(momentum=0.9, epsilon=1e-5, name='gb4')
        
        self.g_resap = Reshape((self.dim, self.dim, self.gen_depth), name='gres')
        self.g_relu = Activation('relu', name='greu')
        self.g_leak = LeakyReLU(self.alpha, name='glk')
        self.g_tanh = Activation('tanh', name='gtan')
        self.g_concat = Concatenate(axis=-1, name='gcon')
        
        #discriminator weights
    
        #32 32 64
        self.d_conv_1 = Conv2D(self.dis_depth*1, 5, strides=2, kernel_initializer=truncate_normal, padding='same', name='dc1') 
        self.d_conv_2 = Conv2D(self.dis_depth*2, 5, strides=2, kernel_initializer=truncate_normal, padding='same', name='dc2')
        self.d_conv_3 = Conv2D(self.dis_depth*4, 5, strides=2, kernel_initializer=truncate_normal, padding='same', name='dc3')
        self.d_conv_4 = Conv2D(self.dis_depth*8, 5, strides=2, kernel_initializer=truncate_normal, padding='same', name='dc4')
        self.d_conv_5 = Conv2D(self.dis_depth*8, 1, strides=1, kernel_initializer=truncate_normal, padding='same', name='dc5')
        
        self.d_leak = LeakyReLU(self.alpha, name='dlk')
        self.d_lamb = Lambda(dick, name='dlm')
        self.d_concat = Concatenate(axis=-1, name='dc')
        self.d_flat = Flatten()
        self.d_dense = Dense(1, kernel_initializer=truncate_normal)
        self.d_sig = Activation('sigmoid')
        
    def getgenerator(self, input_1, input_2):
        
        #generator network
        self.gn_dense_0 = self.s_leak(input_2)
        
        self.gn_concat = self.g_concat([input_1, self.gn_dense_0])
        
        self.gn_dense_1 = self.g_dense_1(self.gn_concat)
        self.gn_batch_1 = self.g_batch_1(self.gn_dense_1)
        self.gn_resap_1 = self.g_resap(self.gn_batch_1)
        
        self.gn_conv_1 = self.g_conv_1(self.gn_resap_1)
        self.gn_batch_2 = self.g_batch_2(self.gn_conv_1)
        self.gn_relu_1 = self.g_relu(self.gn_batch_2)
        
        self.gn_conv_2 = self.g_conv_2(self.gn_relu_1)
        self.gn_batch_3 = self.g_batch_3(self.gn_conv_2)
        self.gn_relu_2 = self.g_relu(self.gn_batch_3)
        
        self.gn_conv_3 = self.g_conv_3(self.gn_relu_2)
        self.gn_batch_4 = self.g_batch_4(self.gn_conv_3)
        self.gn_relu_3 = self.g_relu(self.gn_batch_4)
        
        self.gn_conv_4 = self.g_conv_4(self.gn_relu_3)
        self.gn_tanh = self.g_tanh(self.gn_conv_4)
        
        return self.gn_tanh
        
    def getdiscriminator(self, input_1, input_2):
        
        #discriminator network
        
        #32 32 64
        self.dn_conv_1 = self.d_conv_1(input_1)
        self.dn_leak_1 = self.d_leak(self.dn_conv_1)
        
        #16 16 128
        self.dn_conv_2 = self.d_conv_2(self.dn_leak_1)
        self.dn_leak_2 = self.d_leak(self.dn_conv_2)
        
        #8 8 256
        self.dn_conv_3 = self.d_conv_3(self.dn_leak_2)
        self.dn_leak_3 = self.d_leak(self.dn_conv_3)
        
        #4 4 512
        self.dn_conv_4 = self.d_conv_4(self.dn_leak_3)
        self.dn_leak_4 = self.d_leak(self.dn_conv_4)
        
        self.dn_dense_0 = self.s_leak(input_2)
        self.dn_lamba = self.d_lamb(self.dn_dense_0)
        self.dn_concat = self.d_concat([self.dn_leak_4, self.dn_lamba])
        
        self.dn_conv_5 = self.d_conv_5(self.dn_concat)
        self.dn_leak_5 = self.d_leak(self.dn_conv_5)
        
        self.dn_flat = self.d_flat(self.dn_leak_5)
        
        self.dn_dense = self.d_dense(self.dn_flat)
        return self.dn_dense
        
    def getmodels(self):
        
        input_shape1 = (64, 64, 3)
        input_shape2 = (2400,)
        input_shape3 = (100,)

        self.input_1 = Input(shape=input_shape1) #image
        self.input_2 = Input(shape=input_shape2) #seq vec
        self.input_3 = Input(shape=input_shape3) #random noise
        
        self.generator = self.getgenerator(self.input_3, self.input_2)
        self.discriminator = self.getdiscriminator(self.input_1, self.input_2)
        
        self.generatorModel = Model(inputs=[self.input_3, self.input_2], outputs=[self.generator])
        self.discriminatorModel = Model(inputs=[self.input_1, self.input_2], outputs=[self.discriminator])
        
        self.generatorModel.summary()
        self.discriminatorModel.summary()
        
        self.netD_real_input = Input(shape=(64, 64, 3))
        self.netD_wrong_input = Input(shape=(64, 64, 3))
        self.seq2vec = Input(shape=(2400,))
        self.wrong_seq2vec = Input(shape=(2400,))
        self.noisev = Input(shape=(100,))
        
        self.netD_fake_input = self.generatorModel([self.noisev, self.seq2vec])

        self.epsilon_input = K.placeholder(shape=(None,1,1,1))
        self.epsilont_input = K.placeholder(shape=(None,1))
        self.netD_mixed_input = Input(shape=(64, 64, 3),
            tensor=self.epsilon_input * self.netD_real_input + (1-self.epsilon_input) * (self.netD_fake_input+self.netD_wrong_input + self.netD_real_input)/3)
        
        self.mixed_seq2vec = Input(shape=(2400,),
            tensor=self.epsilont_input * self.seq2vec + (1-self.epsilont_input) * (self.seq2vec + self.seq2vec + self.wrong_seq2vec)/3)
        
        self.loss_real = K.mean(self.discriminatorModel([self.netD_real_input, self.seq2vec]))
        self.loss_wrong = K.mean(self.discriminatorModel([self.netD_wrong_input, self.seq2vec]))
        self.loss_wrong2 = K.mean(self.discriminatorModel([self.netD_real_input, self.wrong_seq2vec]))
        self.loss_fake = K.mean(self.discriminatorModel([self.netD_fake_input, self.seq2vec]))

        self.grad_mixed = K.gradients(self.discriminatorModel([self.netD_mixed_input, self.mixed_seq2vec]), [self.netD_mixed_input])[0]
        self.norm_grad_mixed = K.sqrt(K.sum(K.square(self.grad_mixed), axis=[1,2,3]))
        self.grad_penalty = K.mean(K.square(self.norm_grad_mixed -1))

        self.d_loss = (self.loss_fake + self.loss_wrong + self.loss_wrong2)/3 - self.loss_real + 10 * self.grad_penalty


        self.d_training_updates = Adam(lr=1e-4, beta_1=0.0, beta_2=0.9).get_updates(self.discriminatorModel.trainable_weights,[], self.d_loss)
        self.netD_train = K.function([self.netD_real_input, self.netD_wrong_input, self.seq2vec, self.wrong_seq2vec, self.noisev, self.epsilon_input, self.epsilont_input],
                                [self.d_loss],    
                                self.d_training_updates)
        
        self.g_loss = -self.loss_fake 
        self.g_training_updates = Adam(lr=1e-4, beta_1=0.0, beta_2=0.9).get_updates(self.generatorModel.trainable_weights,[], self.g_loss)
        self.netG_train = K.function([self.noisev, self.seq2vec], [self.g_loss], self.g_training_updates)
        
    def loadseq2vec(self):
        self.seq2vec = np.load('../../../GAN/sequenceVector.npy')
        
    def train(self):
        
        vecp = np.load('test.npy')
        bigdata = np.load('../../../GAN/data2.npy')
        
        for k in range(0, self.epochs):
            
            ite=0
            while ite<len(bigdata):
                data = bigdata[ite:(ite+self.batch)]
                vecs = self.seq2vec[ite:(ite+self.batch)]
                wdata = [data[(x+1)%len(data)] for x in range(len(data))]
                wvecs = [vecs[(x+1)%len(vecs)] for x in range(len(vecs))]
                
                noise = np.random.normal(0, 1.0, size=[len(data), 100])
                e1 = np.random.uniform(0,1,[len(data),1,1,1])
                e2  = np.reshape(e1,[len(data),1])
                for l in range(5):
                    errD,  = self.netD_train([data, wdata, vecs, wvecs, noise, e1, e2])
                    errD = np.mean(errD)

                for l in range(1):
                    errG, = self.netG_train([noise, vecs])
                    errG = np.mean(errG)

                print(errD, errG)
                ite+=self.batch
                
            noise = np.random.normal(0, 1.0, size=[len(data), 100])
            images_fake = self.generatorModel.predict([noise[:36], vecp])
            width = 6
            new_im = Image.new('RGB', (64*width,64*width))
            for ii in range(width):
                for jj in range(width):
                    index=ii*width+jj
                    image = (images_fake[index]/2+0.5)*255
                    image = image.astype(np.uint8)
                    new_im.paste(Image.fromarray(image,"RGB"), (64*ii,64*jj))
            filename = "images_adam/fakeFace%d.png"%k
            new_im.save(filename)
            self.generatorModel.save("model_adam/generator%d.h5"%k)
            self.discriminatorModel.save("model_adam/discriminator%d.h5"%k)
            ite += 1

In [ ]:
dcgan = dcgan()
dcgan.getmodelweight()
dcgan.getmodels()
dcgan.loadseq2vec()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 2400)         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
slk (LeakyReLU)                 (None, 2400)         0           input_2[0][0]                    
__________________________________________________________________________________________________
gcon (Concatenate)              (None, 2500)         0           input_3[0][0]                    
                                                                 slk[0][0]                        
__________

In [ ]:
dcgan.train()

(9.0489817, 0.030910425)
(-4.3572321, -0.071179643)
(-16.558407, 0.023932785)
(-19.522484, -0.47950184)
(-23.361786, -0.69850534)
(-24.42075, -2.4729657)
(-24.207132, -3.3138113)
(-24.585211, -7.2964706)
(-22.196051, -8.62321)
(-21.253311, -14.257074)
(-19.368805, -12.19301)
(-17.66745, -15.422531)
(-17.017715, -3.9236243)
(-16.651588, -11.330822)
(-18.051504, 13.463148)
(-18.214447, 0.12642547)
(-17.897833, 17.632111)
(-16.707352, -4.8889828)
(-19.67115, 12.473589)
(-17.478252, -13.185506)
(-19.034645, 13.579783)
(-17.823647, -17.382923)
(-17.94124, 11.63826)
(-17.968887, 4.1644287)
(-18.10961, 20.863174)
(-18.440062, -10.262674)
(-18.90888, 7.6024804)
(-18.346333, -11.422801)
(-19.509457, 12.115822)
(-17.942888, -7.8165607)
(-17.303419, 15.71217)
(-17.555454, -15.221981)
(-22.006239, 23.128357)
(-20.474102, -1.4258374)
(-19.192684, 10.883112)
(-20.545322, -13.690836)
(-18.534367, 3.7455142)
(-19.137747, -16.211109)
(-20.269798, 11.344815)
(-18.242058, -11.834137)
(-19.311216, 8.69326

(-11.223014, -38.590248)
(-11.205891, -17.257126)
(-11.240829, -21.679131)
(-12.350232, 1.5328078)
(-11.125927, -22.183794)
(-12.145834, -18.954664)
(-10.894546, -16.624439)
(-12.19301, -8.1859694)
(-13.156145, -34.081612)
(-11.019066, -25.254637)
(-11.592476, 2.1443615)
(-12.171041, -5.5048261)
(-10.882686, -22.500271)
(-11.521115, -38.838268)
(-11.488876, 3.7335365)
(-12.34399, 2.3415003)
(-11.261419, -22.642258)
(-10.77465, -26.809433)
(-11.067733, -17.715836)
(-10.485864, -48.384186)
(-11.559759, -6.959074)
(-11.338337, -20.669203)
(-11.089533, -21.107443)
(-11.960363, -10.885523)
(-11.687202, -1.3258671)
(-12.908545, -16.607237)
(-11.93461, 3.0951841)
(-11.249441, -18.911882)
(-13.105783, -14.314543)
(-11.565469, -18.839167)
(-11.010951, -6.620029)
(-13.340633, -28.022289)
(-14.197064, 3.4347486)
(-12.170933, -20.599237)
(-12.887556, 2.0998254)
(-10.764749, -25.617691)
(-11.642266, -28.538116)
(-11.43835, -28.460617)
(-11.696322, -35.034119)
(-11.656097, -12.323267)
(-11.133772, -